# Testing Individual components of the FV HE scheme

In [1]:
from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encrypter import Encrypter
from syft.frameworks.torch.he.fv.decrypter import Decrypter
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/ravikantsingh/Desktop/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'



## Generates coeff_modulus using Polymodulus degree and bit-length parameters.
Generates random integer of the given bit length for using as coeff_modulus

In [2]:
params = EncryptionParams()
cm = CoeffModulus()
params.coeff_modulus = cm.create(4096, [30, 50, 60])
print("coeff_modulus: ", params.coeff_modulus)

coeff_modulus:  [1073692673, 1125899906826241, 1152921504606830593]


## Use standard values of coeff_modulus for the given Poly Modulus degree and security_level.
Need some refractoring and integration of code.

In [3]:
params_standard = EncryptionParams()
cm_standard = CoeffModulus()
for i in SeqLevelType:
    for j in [1024, 4096, 32768]:           # ... [1024, 2048, 4096, 8192, 16384, 32768]
        params_standard.coeff_modulus = cm_standard.bfv_default(j, i)
        print(f"coeff_modulus for {i} and poly_modulus = {j}", params_standard.coeff_modulus)
    print('\n')

coeff_modulus for SeqLevelType.TC128 and poly_modulus = 1024 [132120577]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 4096 [68719403009, 68719230977, 137438822401]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 32768 [36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193, 36028797012606977, 36028797010444289, 36028797009985537, 36028797005856769, 36028797005529089, 36028797005135873, 36028797003694081, 36028797003563009, 36028797001138177, 72057594037338113]


coeff_modulus for SeqLevelType.TC192 and poly_modulus = 1024 [520193]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 4096 [33538049, 33349633, 33292289]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 32768 [18014398506729473, 18014398505943041, 18014398499848193, 18014398498799617, 18014398498275329, 36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193]


coeff_modulus for Se

## Keygeneration

In [26]:
params = EncryptionParams(4096, CoeffModulus().bfv_default(2048, SeqLevelType.TC192) , 64 )
ctx = Context(params)
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [27]:
print('secret key values : ', sk.data)
print(sk)

secret key values :  tensor([           1, 137438691328,            1,  ...,            0,
        137438691328,            1])


In [28]:
print(pk.data[1])

tensor([  3273678555, 101679961848, 116760775903,  ...,  72926444803,
        100612310014,  55408122781])


In [29]:
print('public key values : ', pk.data)

public key values :  [[tensor(134165012779), tensor(50811248882), tensor(20677915423), tensor(1), tensor(22340007815), tensor(5976368602), tensor(50328482210), tensor(0), tensor(56701249814), tensor(11032820549), tensor(137438691326), tensor(137438691324), tensor(137438691328), tensor(1), tensor(81970829794), tensor(1), tensor(16763267637), tensor(4), tensor(137438691326), tensor(0), tensor(82152346982), tensor(4), tensor(106325606721), tensor(0), tensor(137438691328), tensor(17496752795), tensor(41748244398), tensor(137438691328), tensor(18243027749), tensor(0), tensor(101320314641), tensor(17228206236), tensor(2), tensor(8268805953), tensor(5), tensor(71155018627), tensor(116158112314), tensor(132460806992), tensor(64318085308), tensor(30777723170), tensor(103046577150), tensor(33645421664), tensor(48898597796), tensor(7455839797), tensor(4), tensor(52837759084), tensor(135932881935), tensor(44741414342), tensor(137438691327), tensor(137438691327), tensor(20554052447), tensor(5373020

## Integer Encoder
Encodes Integer values to Plaintext object

In [30]:
int_encoder = IntegerEncoder(ctx)
plaintext = int_encoder.encode(100)
print(plaintext)
print('plaintext data',plaintext.data)

plaintext data [0, 0, 1, 0, 0, 1, 1]


### Decodes back to Integer

In [31]:
print(int_encoder.decode(plaintext))

100


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [32]:
encrypter = Encrypter(ctx, pk)
ciphertext = encrypter.encrypt(plaintext)
print(ciphertext)
print('ciphertext data :', ciphertext.data)

plain_0 <class 'list'>
ciphertext data :[[tensor(76825237208), tensor(2), tensor(3), tensor(3), tensor(22340007814), tensor(3), tensor(137438691326), tensor(137438691321), tensor(109057490150), tensor(131908818615), tensor(3), tensor(0), tensor(0), tensor(3), tensor(81970829794), tensor(137438691327), tensor(16763267637), tensor(137438691326), tensor(137438691327), tensor(2), tensor(96357125789), tensor(1), tensor(84263507648), tensor(137438691328), tensor(0), tensor(0), tensor(116561383246), tensor(68719738881), tensor(0), tensor(0), tensor(86785762290), tensor(137438691326), tensor(2), tensor(133330650820), tensor(137438691327), tensor(101851506305), tensor(137438691328), tensor(132460806994), tensor(0), tensor(30777723167), tensor(103046577147), tensor(120637658786), tensor(48898597800), tensor(7455839795), tensor(137438691326), tensor(52837759084), tensor(137438691323), tensor(44741414347), tensor(137438691327), tensor(137438691327), tensor(20554052451), tensor(134750035669), tenso

In [33]:
# TODO Need some changes.

# encrypter_symmetric = Encrypter(ctx, sk)
# ciphertext_symmetric = encrypter_symmetric.encrypt(plaintext)
# print(ciphertext_symmetric)

Encrypt Plaintext to ciphertext using secret_key

## Decrypter
Decrypt Ciphertext to Plaintext using secret_key

In [34]:
# Waiting for SEAL v3.5

# decrypter = Decrypter(ctx, sk)
# result = decrypter.decrypt(ciphertext)
# print("len : ",len(result.data))
# print(result.data)
# print(result)

In [35]:
# print(int_encoder.decode(result))